# Test 1

Creating a 2D playground

This approach will not work. Too much trouble to compute gradients after integration.\
Make everything numerically.

In [86]:
import numpy as np
import sympy as smp
import matplotlib.pyplot as plt
from numba import jit
from scipy import integrate
import numdifftools as nd

Some parameter definitions. Change this cell to put a different number of parameters

In [99]:
# Symbol definition
lmb, x, y, = smp.symbols("\lambda x y", real=True)
#A = smp.symbols("A_1 A_2 A_3 A_4", real=True)
A = smp.symbols("A_1 A_2", real=True)
U, med = smp.symbols("U, NL", cls=smp.Function)

U = U(x,y,*A)
med = med(U,x,y)

lagrangian_density = lmb*smp.Abs(U)**2 + sum(smp.diff(U,i)**2 for i in [x,y] ) + med

# Some media
kerr = smp.Abs(U)**4/2

In [100]:
ansatz = A[0]*smp.exp(-(x**2+y**2)/A[1]) #+ A[2]*smp.exp(-((x-1)**2+y**2)/A[3]) 
ansatz

A_1*exp((-x**2 - y**2)/A_2)

In [101]:
substitutions = {
    U:ansatz,
    med:kerr,
    lmb:1
}
density = lagrangian_density.subs(substitutions).doit()
density

A_1**4*exp(-4*(x**2 + y**2)/A_2)/2 + A_1**2*exp(-2*(x**2 + y**2)/A_2) + 4*A_1**2*x**2*exp(2*(-x**2 - y**2)/A_2)/A_2**2 + 4*A_1**2*y**2*exp(2*(-x**2 - y**2)/A_2)/A_2**2

In [102]:
def get_lagrangian_value(params_vals,density):
    params_subs = {A[i]:params_vals[i] for i in range(len(params_vals))}
    space_density = smp.lambdify([x,y],density.subs(params_subs))
    return integrate.dblquad(space_density,-np.inf, np.inf, -np.inf, np.inf)[0]